Consulta 03
Esta consulta devuelve devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

Importar las librerías necesarias para el procesamiento de los datos

In [1]:
import pandas as pd


importar archivos


In [2]:
games = pd.read_csv(r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\ETL\games_clean.csv')
df_items = pd.read_parquet(r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\PARQUET\df_items_clean.parquet')

In [3]:
df_games = pd.DataFrame(games)

In [4]:
df_itemscopy = df_items.copy()
df_gamescopy = df_games.copy()

In [7]:

df_itemscopy

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
1,76561197970982479,277,20,0.0
2,76561197970982479,277,30,7.0
3,76561197970982479,277,40,0.0
4,76561197970982479,277,50,0.0
...,...,...,...,...
5170010,76561198329548331,7,373330,0.0
5170011,76561198329548331,7,388490,3.0
5170012,76561198329548331,7,521570,4.0
5170013,76561198329548331,7,519140,3.0


In [8]:
df_gamescopy

,genres,app_name,release_date,tags,price,id,developer
0,Action,Lost Summoner Kitty,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,761140.0,Kotoshiro
1,Indie,Ironbound,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",NaN,643980.0,Secret Level SRL
2,Casual,Real Pool 3D - Poolians,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",Free,670290.0,Poolians.com
3,Action,弹炸人2222,2017-12-07,"['Action', 'Adventure', 'Casual']",0.99,767400.0,彼岸领域
4,Action,Log Challenge,NaT,"['Action', 'Indie', 'Casual', 'Sports']",2.99,773570.0,NaN
...,...,...,...,...,...,...,...
32130,Casual,Colony On Mars,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",1.99,773640.0,"Nikita ""Ghost_RUS"""
32131,Casual,LOGistICAL: South Africa,2018-01-04,"['Strategy', 'Indie', 'Casual']",4.99,733530.0,Sacada
32132,Indie,Russian Roads,2018-01-04,"['Indie', 'Simulation', 'Racing']",1.99,610660.0,Laush Dmitriy Sergeevich
32133,Casual,EXIT 2 - Directions,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",4.99,658870.0,"xropi,stev3ns"


Realizamos las siguientes acciones para hacer la consulta que necesitamos:

Verificar si la columna 'genres' está presente en el DataFrame df_games_copy.
Convertir la columna 'release_date' a tipo datetime.
Filtrar el DataFrame df_games_copy por el género proporcionado.
Unir el DataFrame filtrado con df_items_copy utilizando la columna 'id' en df_games_copy y 'item_id' en df_items_copy.
Calcular las horas jugadas por cada usuario para cada juego.
Encontrar el usuario que ha jugado más horas en total.
Calcular la acumulación de horas jugadas por año de lanzamiento para el género dado.
Retornar un diccionario con el usuario que ha jugado más horas para el género dado y la acumulación de horas jugadas por año de lanzamiento.

In [10]:
def UserForGenre(genero: str, df_games_copy, df_items_copy):
    # Verificar si la columna 'genre' está presente en el DataFrame df_games_copy
    if 'genres' not in df_games_copy.columns:
        raise ValueError("El DataFrame df_games_copy no tiene una columna llamada 'genre'.")

#Convertir la columna 'release_date' a tipo datetime
    df_games_copy['release_date'] = pd.to_datetime(df_gamescopy['release_date'], errors='coerce')

#Filtrar df_games_copy por el género dado
    juegos_genero = df_games_copy[df_games_copy['genres'] == genero]

#Unir el DataFrame filtrado con df_user_items
    juegos_usuario = juegos_genero.merge(df_itemscopy, left_on='id', right_on='item_id')

#Calcular las horas jugadas por usuario para cada juego
    horas_por_usuario = juegos_usuario.groupby('user_id')['playtime_forever'].sum().reset_index()

#Encontrar el usuario con más horas jugadas
    usuario_max_horas = horas_por_usuario.loc[horas_por_usuario['playtime_forever'].idxmax()]['user_id']

#Calcular la acumulación de horas jugadas por año de lanzamiento para el género dado
    horas_por_año = juegos_usuario.groupby(juegos_usuario['release_date'].dt.year)['playtime_forever'].sum().reset_index()
    horas_por_año.rename(columns={'playtime_forever': 'Horas'}, inplace=True)
    horas_por_año = horas_por_año.to_dict('records')

#Crear el diccionario de resultados
    result = {
        "Usuario con más horas jugadas para {}: ".format(genero): usuario_max_horas,
        "Horas jugadas": horas_por_año
    }

    return result

In [11]:
genero = input("Ingrese el nombre del género: ")
resultado = UserForGenre(genero, df_gamescopy, df_itemscopy)
print(resultado)

ValueError: You are trying to merge on float64 and object columns for key 'id'. If you wish to proceed you should use pd.concat